In [1]:
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy

from statsmodels.sandbox.regression.gmm import GMM
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge

In [2]:
data = pd.read_stata('chile.dta')
data = data.set_index(['id','year'])

In [3]:
ldata = data.copy()

ldata[['routput', 'totlab', 'renerg','realmats',
       'rcapstock']] = np.log(data[['routput','totlab', 'renerg','realmats','rcapstock']])

ldata = ldata.dropna()

do a non-parametric regression with $l_{it}$, $m_{it}$, $k_{it}$ on $y_{it}$ and save residual, and fitted value

given a guess of $\beta_1$ and $\beta_2$ compute an implied omega i.e. productive shock
get wit-1 (i.e. without estimating a function)
the residuals should be exogenous
these are the moment conditions

https://www.aeaweb.org/content/file?id=3015

In [4]:
d = ldata['routput'],ldata[['totlab']],ldata[['rcapstock','realmats','renerg']]

def np_resids(y,x):
    """residuals from lasso"""
    poly = PolynomialFeatures(degree=3)
    x_poly = poly.fit_transform(x)
    clf = Ridge(alpha=0.0)
    clf.fit(x_poly, y) 
    resid = y-clf.predict(x_poly)
    return resid

stage1_resids = np_resids(ldata['routput'],ldata[['totlab','rcapstock','realmats','renerg']])

In [7]:
def stage2obj(params, stage1_resids, dta):
    y,x = dta
    
    #compute value addeed to production by shock and all terms
    value = y - stage1_resids
    
    #compute implied value of endogenous shock
    shock = value - x.mul(params,axis=1).sum(axis=1)
    
    #####compute expected production trend############
    shocklag = shock.groupby('id').shift(1)
    both_shocks = pd.concat((shock, shocklag),axis=1).dropna()
    
    ###### save residuals ########
    shock2 = np_resids(both_shocks[[0]], both_shocks[[1]])[0]
    
    #####multiply by lags to get moment condition #########
    #moment1 = x['rcapstock'].mul(shock2,axis=0)
    #moment2 = x['totlab'].groupby('id').shift(1).mul(shock2,axis=0)
    #moments = pd.concat((moment1,moment2),axis=1)
    moments = x.mul(shock2,axis=0)
    return moments.dropna()
    
d = ldata['routput'],ldata[['rcapstock','totlab']]
stage2obj(np.array([.1,.1]), stage1_resids, d)

rcapstock    totlab
id       year                     
18.0     1980   1.145648  0.356509
         1981  -5.070534 -1.357924
54.0     1980   0.879464  0.370622
         1981  -4.311159 -1.599216
         1982   0.866176  0.265023
         1983  -3.241297 -0.945828
         1984  -0.285848 -0.085370
         1985  -1.034154 -0.357779
         1986   1.320648  0.463455
         1987  -2.971930 -0.922297
126.0    1989  -2.551964 -1.343777
         1990   5.390442  3.426523
         1991  -3.463474 -1.779423
         1992   0.508464  0.260784
         1993  -1.882293 -0.933710
         1994   1.835517  0.924008
         1995  -1.126449 -0.567183
504.0    1980   0.959151  0.321435
         1981  -1.658002 -0.643922
         1982   1.336386  0.574954
         1983  -2.888035 -1.151569
513.0    1982  -1.209020 -0.794167
         1983  -6.611845 -2.122324
         1984  -8.641200 -2.514537
         1985  -6.753154 -1.976067
         1986 -10.562725 -2.227811
         1987  -0.868642 -0.278003
         1988  -9.029393 -2.407305
522.0    1980  -1.115309 -0.338332
         1981   0.267605  0.087723
...                  ...       ...
251235.0 1996   4.358946  1.908248
251388.0 1996  -4.868701 -1.633427
251433.0 1996  -0.157720 -0.052909
251478.0 1996   0.479690  0.200229
251676.0 1996  -0.389776 -0.161375
251712.0 1996   1.370273  0.412455
251748.0 1996   2.528896  0.960937
251829.0 1996   3.008699  1.414907
251838.0 1996   1.373774  0.709260
251847.0 1996  -0.559730 -0.383685
251874.0 1996   2.343864  0.960652
251883.0 1996  -1.231506 -0.593789
252081.0 1996  -0.507913 -0.180955
252207.0 1996   1.899208  0.578463
252315.0 1996  -1.336254 -0.630105
252324.0 1996  12.329204  5.895966
252513.0 1996  -0.492174 -0.197801
252666.0 1996  -1.001818 -0.346829
252882.0 1996   7.144176  2.611936
252918.0 1996  -3.220582 -1.103471
252963.0 1996  -3.246163 -1.321640
252972.0 1996   7.288294  2.749966
252981.0 1996  -8.205707 -2.488737
253215.0 1996   8.084631  3.258724
253224.0 1996   4.233215  1.902376
253269.0 1996  10.692027  4.412790
253341.0 1996  -0.455200 -0.161348
253431.0 1996   2.217087  0.713309
253593.0 1996   8.377827  2.376548
253629.0 1996  -2.735315 -1.661019

[18044 rows x 2 columns]

In [8]:
class ACF(GMM):
    
    def __init__(self, dta, stage1_resid, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        y,x = dta
        super(ACF, self).__init__(y,x,stage1_resid, x.shape[1],*args, **kwds)
        self.endog = y
        self.exog = x
        self.instr = stage1_resids
        
    def momcond(self, params):
        d = self.endog, self.exog
        return np.array( stage2obj(params, self.instr , d) )
        
        
d = ldata['routput'],ldata[['rcapstock','totlab']]
acfmodel = ACF(d, stage1_resids)
acfresult = acfmodel.fit(np.array([.1,.5]),optim_method='nm')
print acfresult.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 44
         Function evaluations: 84
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 19
         Function evaluations: 39
                                 ACF Results                                  
Dep. Variable:                routput   Hansen J:                    2.283e-08
Model:                            ACF   Prob (Hansen J):                   nan
Method:                           GMM                                         
Date:                Tue, 14 May 2019                                         
Time:                        16:22:01                